### Library import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from utlis import Error_cal, Data_loader
import warnings
import config as c
from model import PsRMFNN
import scipy.io
import numpy as np
from sklearn.model_selection import KFold, LeaveOneOut #K交叉，留一法
# For evaluating our ML results
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # 标准化
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')
%matplotlib inline

tf.reset_default_graph()   # To clear the defined variables and operations of the previous cell

### Data preparation

In [2]:
x_train, x_test, ytrain, ytest = Data_loader('data_x.csv', 'data_y.csv', 0.2)

In [3]:
y_exp_train = ytrain[:, :1]
y_exp_test = ytest[:, :1]
y_low_train = ytrain[:, -1:]
y_low_test = ytest[:, -1:]

### Model training

In [4]:
model = PsRMFNN(x_train, c.layers_low, c.layers_high, c.layers_linear, y_exp_train, 30, c.lr, y_low_train)
model.train(c.epoch)






Device mapping:




([0.99548936,
  0.98768055,
  0.9804434,
  0.9614685,
  0.9404445,
  0.9313643,
  0.8472254,
  0.8544653,
  0.83197814,
  0.7225544,
  0.6514808,
  0.6026707,
  0.60173064,
  0.53676236,
  0.6743971,
  0.60629493,
  0.5967782,
  0.55633557,
  0.5448539,
  0.40128675,
  0.4130806,
  0.4528391,
  0.44905713,
  0.46148092,
  0.43095008,
  0.39791578,
  0.27735478,
  0.37689012,
  0.32517576,
  0.26820138,
  0.32505003,
  0.28700468,
  0.26476988,
  0.35767567,
  0.27645865,
  0.18962762,
  0.2884741,
  0.2227547,
  0.26219785,
  0.22078165,
  0.18070456,
  0.23451258,
  0.23604017,
  0.18810615,
  0.15423848,
  0.1532958,
  0.121466205,
  0.16316962,
  0.26214904,
  0.10596873,
  0.2616094,
  0.10071467,
  0.13072947,
  0.15131086,
  0.13009329,
  0.11088653,
  0.0981097,
  0.0662983,
  0.15179887,
  0.11684034,
  0.075577594,
  0.100595094,
  0.12151604,
  0.11393657,
  0.081277825,
  0.087005086,
  0.070967235,
  0.12511747,
  0.11088649,
  0.08296616,
  0.08389974,
  0.08999159,
  0.07

### Evaluation

In [5]:
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

In [6]:
Err_train = Error_cal(y_train_pred, y_exp_train)
Err_test = Error_cal(y_test_pred, y_exp_test)
metrics = np.concatenate((np.reshape(Err_train, (1, 6)), np.reshape(Err_test, (1, 6))), 0)
df = pd.DataFrame(metrics, columns=['R2', 'MAE', 'RMSE', 'MAPE', 'Mean', 'Cov'], index=['train', 'test'])

### Results save

In [7]:
Train_pred = np.concatenate((y_train_pred[0,:,:], y_exp_train), 1)
Test_pred = np.concatenate((y_test_pred[0,:,:], y_exp_test), 1)
data_frame_train = pd.DataFrame(Train_pred, columns=['PsRMFNN', 'exact'])
data_frame_test = pd.DataFrame(Test_pred, columns=['PsRMFNN', 'exact'])

In [8]:
writer = pd.ExcelWriter('Prediction.xlsx')
data_frame_train.to_excel(writer, 'Train', float_format='%.15f', index=False)
data_frame_test.to_excel(writer, 'Test', float_format='%.15f', index=False)
df.to_excel(writer, 'metrics', float_format='%.15f', index=False)
writer.save()
writer.close()